In [1]:
#Allows relative imports
import os
import sys
import pandas as pd 

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
#imports 
from src.preprocessing import *
from src.models import *
from src.train_eval_helpers import *
from src.plots import *
import torch
import torch.nn as nn
import torch.nn.functional as F
%load_ext autoreload
%autoreload 2
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.dpi']= 300
import seaborn as sns
sns.set_style('darkgrid')

#checking gpu status
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("Using : {}".format(device))
else:
    device = torch.device('cpu')
    print("Using : {}".format(device))

RANGE = range(12,17)
TRAINDIR = '../TrainingData/'

Using : cuda


### Parsing Naïve TCR db


In [2]:
epitope_map = {
    "ELAGIGILTV" : 1 #Melanoma MLANA
    ,"GILGFVFTL"  : 0 #Influenza A M gene
    ,"SLFNTVATLY" : 0 #HIV
    ,"RMFPNAPYL" : 1 #WT1
    ,"LLDFVRFMGV" : 1 #EBNA-B EBV but also linked to 
    ,"GLCTLVAML" : 1 #BMLF1 ebv
    ,"LLFGYPVYV" : 0 #HTLV-1 Tax
    ,"SLLMWITQV" : 1 #well known squamous cell NY-ESO
    ,"FLYALALLL" : 1 #EBV LMP2A, but linked to lymphoma
    ,"FLASKIGRLV" : 0 # PLA2G6 phospholipase
    ,"MLDLQPETT" : 1 #HPV E7
    ,"IMDQVPFSV" : 1 #PMEL melanosome....
    ,"YLLEMLWRL" : 1 #EBV lmp1 viral oncogene
    ,"KVLEYVIKV" : 1 #MAGE A1
    ,"RTLNAWVKV" : 0 #HIV gag1
    ,"KTWGQYWQV" : 1 #PMEL
    ,"YLNDHLEPWI" : 1 #BCL-2-l1
    ,"SLFNTVATL" : 0 #HIV Gag 1
    ,"CLLWSFQTSA" : 0 #tyr (ALBINISM)
    ,"ILKEPVHGV" : 0 #HIV1 POL
    ,"CLLGTYTQDV" : 0 #KANAMYCYN KANJ (ANTIBIO)
    ,"NLVPMVATV" :  0 #H CMV
    ,"KLQCVDLHV" : 0 # klk3
    ,"ELAGIGLTV" : 1 #MLANA
    ,"VLEETSVML" : 0 #CMV IE1
    ,"YVLDHLIVV" :  0 #HPV brlf1
    ,"FLGKIWPSHK" : 0 # HIV gag1
    ,'KVAELVHFL' : 1 #melanoma associated antigen 9
    ,"CLNEYHLFL" : 0 # human NDC1
    ,"AMFWSVPTV" : 0 #TKT phosphate glycolysis 
    ,"FLYNLLTRV" :  0 #SEC24A
    ,"KLMNIQQKL" : 1 #AKAP13
    ,"IILVAVPHV" : 0 #EXOC8
    ,"MLGEQLFPL" : 0 #PABPC1
    ,"YILEETSVM" : 0 # CMV IE1
    ,"CINGVCWTV" :  1 #NS3 Hep C??
    ,"KLSALGINAV" : 1 #NS3 Hep C??
    ,"LLWNGPMAV" :  1 #NS4B Hep C??
    ,"TLNAWVKVV" : 0 #HIV GAG1
    ,"KINAWIKVV" : 0 #HIV gag
    ,"SLYNTVATL" : 0 #HIV gag
    ,"CVNGSCFTV" : 0 #Influenza A
    ,"ARNLVPMVATVQGQN" : 0 #pp65
    ,"LSEFCRVLCCYVLEE" : 0 #CMV IE1
    ,"LLFGYAVYV" : 0 # HLTV tax
    ,"LLFGYPRYV" : 0 #HLTV tax
    ,"SLLMWITQC" : 1 #NY-ESO-1
    ,"LLFGKPVYV" : 0 #HLTV tax
    ,"LLFGPVYV" : 0 #HLTV tax
    ,"MLWGYLQYV" : 0 #HLTV tax
    ,"LGYGFVNYI" : 0 #TAX
    ,"AAGIGILTV" : 1 #MLANA
    ,"LLFGFPVYV" : 0 #tax
    ,"ALWGPDPAAA" : 0 #Insuline INS
    ,"ELAAIGILTV" : 1 #MLANA
    ,"ELAGIGALTV" : 1 #MLANA
    ,"ILAKFLHWL" : 1 #TERT
    ,"EAAGIGILTV" : 1 # MLANA
    ,"LLFGYPVAV" : 0 #Tax HTLV
    ,"ALGIGILTV" : 1 #MLANA
    ,"LLLGIGILV" : 0 #BST2
    ,"KLVALGINAV" : 1 #NS3 hepc
    ,"CLGGLLTMV" : 1 #LMP2A
    ,"NLSALGIFST" : 0#IGF2BP2
    ,"MLNIPSINV" :0#pp65
    ,"VAANIVLTV" : 0#SLC30A8
    ,'SLYNTVATLY':0 #HIV gag
}

In [4]:
AAs= ['A','R','N','D','B','C','E','Q','Z','G',
'H','I','L','K','M','F','P','S','T','W','Y','V']
def filter_df(df, v = False):
    df = df.dropna(subset=['amino_acid'])\
           .query('amino_acid.str.len()>=12 & amino_acid.str.len()<=16')\
           .query("amino_acid.str.startswith('C') & amino_acid.str.endswith('F')")\
           .query('not amino_acid.str.contains("\*")')\
           .query('not amino_acid.str.contains("\~")')
    df['len'] = df.amino_acid.str.len()
    df.drop_duplicates(subset='amino_acid',inplace=True)
    return df

#manual 1hot encode lol
AA_KEYS = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
           'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
char_to_int = dict((c,i) for i,c in enumerate(AA_KEYS))
int_to_char = dict((i,c) for i,c in enumerate(AA_KEYS))

def onehot_aa(seq):
    int_encoded = [char_to_int[char] for char in seq]
    onehot_encoded = list()
    for value in int_encoded:
        letter = [0 for _ in range(len(AA_KEYS))]
        letter[value] = 1
        onehot_encoded.append(letter)
    return np.array(onehot_encoded).flatten()

from sklearn.decomposition import PCA 

def get_reduc(df, n=3):#, top=100):
    #Getting onehot features for PCA
    pca = PCA(n_components=n)
    tmp = df.copy()
    X = np.array([x for x in df['oh']])
    print(X.shape, type(X))
    comp = pca.fit_transform(X)
    print("Explained var ratios :",pca.explained_variance_ratio_)
    print("Explained var : ",pca.explained_variance_)
    for i in range(n):
        tmp['PCA'+str(i+1)]=comp[:,i]
    var_ratio = pca.explained_variance_ratio_.tolist()
    var_ratio = ['{:g}'.format(float('{:.3g}'.format(i))) for i in var_ratio]
    tmp['var_ratios'] = [var_ratio for _ in range(len(tmp))]
    return tmp

def get_pca(df,n=3):
    df = df.copy()
    df['oh'] = df['amino_acid'].apply(onehot_aa)
    output = get_reduc(df,n)
    return output

def plot_pca(df, hue,name):
    ll = df.len.unique().item()
    vars_=[z for z in df.columns if z.startswith('PCA')]
    g = sns.PairGrid(df, x_vars = vars_,
                     y_vars = vars_,
                     diag_sharey=False, hue=hue)
                     #hue_kws = {'markers':['x','x','+','+']})
    g.fig.suptitle('{} TCR, PCA with {} components for L = {}'\
                   '\nExplained variance ratios : {}'.format(name,len(vars_),ll, 
                                                                 df.var_ratios.values[0]), fontweight='bold')
    g.fig.subplots_adjust(top=0.92)
    g.map_lower(sns.scatterplot, style = df['label'],
                markers=['X','X','P','P'],
                sizes = [0.2,0.2,0.2,0.1], alpha=0.75)#,0.75,0.75,0.5])
    g.map_upper(sns.kdeplot)
    g.map_diag(sns.kdeplot)
    g.add_legend()
    plt.savefig('{}_tcr_PCA_{}comp_L{}.jpg'.format(name, len(vars_),ll), dpi=300)
    
sns.set_palette("husl", n_colors=4)

### Subsampling naive set

In [5]:
sample_df = pd.read_csv('../training_data_new/training_data_curated/sampled_10p_naive_db.csv')

In [15]:
from itertools import product
trbv = sample_df.TRBV.unique()
trbj = sample_df.TRBJ.unique()
sample_df = filter_df(sample_df)
combo = product(trbv,trbj)
#result df
i=0
total = 754
naive_train = pd.DataFrame()
naive_test = pd.DataFrame()
for V,J in combo:  
    print('{:.2%} done'.format(i), end='\r')
    i+=1/total
    #Sampling combo and shuffling
    sub = sample_df.query('TRBV==@V and TRBJ==@J').sample(frac=1)
    for l in range(12,17):
        #Getting tmp of that len, then splitting into two to ensure there's no 
        #overlap
        tmp = sub.query('len==@l')
        tmp_train = tmp[0:math.floor(len(tmp)/2)]
        tmp_test = tmp[math.floor(len(tmp)/2):-1]
        naive_train = naive_train.append(tmp_train.sample(frac=0.2), ignore_index=True)
        naive_test = naive_test.append(tmp_test.sample(frac=0.1), ignore_index=True)
print("train naive sample size : ", len(naive_train))
print("test naive sample size : ", len(naive_test))


train naive sample size :  37917
test naive sample size :  18738


### Cancer dataset

Their healthy samples should only come from Emerson et al. 2017 


Since there are so few in McPas and db_pos, make an aggregate of maximum of cancer sequences as possible. 

First, adding McPas and DB_pos, then checking with training/testing datasets of tumor samples.

In [12]:
mcpas = pd.read_csv('../training_data_new/new_data/McPas-TCR.csv', 
                    usecols = ['CDR3.beta.aa', 'Species', 'Category', 'Epitope.peptide', 
                               'Epitope.ID', 'MHC','Tissue','TRBV', 'TRBD', 'TRBJ'])\
          .rename(columns={'CDR3.beta.aa':'amino_acid'})\
          .query('Species=="Human"')
mcpas = filter_df(mcpas)
mcpas['label'] = mcpas.apply(lambda x : 1 if x['Category'] == 'Cancer' else 0, axis=1)
mcpas_cancer = mcpas.query('Species=="Human"&Category=="Cancer"')

db_pos = pd.read_csv('../training_data_new/new_data/db_pos.csv',
                     usecols = ['epitope','cdr3_TRB','TRBV','TRBJ'])\
           .rename(columns={'cdr3_TRB':'amino_acid'})
db_pos = filter_df(db_pos)
db_pos['label'] =db_pos['epitope'].apply(lambda x: epitope_map[x])
db_pos_cancer = db_pos.query('label==1')

C:\Users\richi\Anaconda3\envs\pdm\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (12,13,14,22,23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [13]:
len(db_pos_cancer)+len(mcpas_cancer)

4313

### Architecture

In [54]:
class xd(torch.nn.Module):
    """
    PyTorch version of DeepCAT CNN
    seq_len represents the length (# of AAs) of the CDR3 region (L = 12, 13, ..., 16)
    When initializing CNN_CDR3, the L should be specified 
    """ 
    def __init__(self, seq_len):
        super(xd, self).__init__()
        #Convolutions
        self.length = seq_len
        self.conv1 = nn.Conv2d(1, 8, kernel_size=(5,4))
        self.pool1 = nn.MaxPool2d(kernel_size = (1,2), stride=(1,1))
        self.conv2 = nn.Conv2d(8, 16, kernel_size = (1,2))
        self.pool2 = nn.MaxPool2d(kernel_size = (1,2), stride=(1,1))
        #Getting the dimension after convolutions
        self.dummy_param = nn.Parameter(torch.empty(0))
        self.name = 'var_atchley_net_'+str(seq_len)
        #Linear/Dense layers
        #self.fc1 = nn.Linear(16*4*2, 10)
        self.fc1 = nn.Linear(16*(seq_len-6), 50)
        self.bn1 = nn.BatchNorm1d(50)
        self.fc2 = nn.Linear(50,10)
        self.bn2 = nn.BatchNorm1d(10)
        self.fc3 = nn.Linear(10,2)
        self.dropout= nn.Dropout(0.4)
            
    def reset_parameters(self):
        for layer in self.children():
            if hasattr(layer, 'reset_parameters'):
                layer.reset_parameters()
                layer.zero_grad()
                
    def forward(self, x):
        #Conv -> ReLU -> MaxPool
        print("in", x.shape)
        x = F.relu(self.conv1(x))
        print("after conv1",x.shape)
        x = self.pool1(x)
        print("after pool1", x.shape)
        x = F.relu(self.conv2(x))
        print("after conv2", x.shape)
        x = self.pool2(x)
        print("after pool2", x.shape)
        #Linear->ReLU->Dropou
        x = x.view(-1, x.shape[1]*x.shape[2]*x.shape[3]) #reshaping after convolution
        print("after reshape", x.shape)
        x = self.dropout(self.bn1(F.relu(self.fc1(x)))) 
        x = self.dropout(self.bn2(F.relu(self.fc2(x)))) 
        x = self.dropout(F.relu(self.fc3(x)))
        
        predictions = x.argmax(1)
        probabilities = x.softmax(1)

        return x, predictions, probabilities

In [66]:
z=nn.Sequential(nn.Linear(100,20),
              nn.BatchNorm1d(20),
              nn.Linear(20,50))

In [68]:
z(torch.empty((1000,100))).shape

torch.Size([1000, 50])

In [69]:
L = 12
X = torch.empty((100,1,5,L))
mod = xd(L)

mod(X)[0].shape

in torch.Size([100, 1, 5, 12])
after conv1 torch.Size([100, 8, 1, 9])
after pool1 torch.Size([100, 8, 1, 8])
after conv2 torch.Size([100, 16, 1, 7])
after pool2 torch.Size([100, 16, 1, 6])
after reshape torch.Size([100, 96])


torch.Size([100, 2])

python train.py -indir .\training_data_new\dataset4\ -arch xd -enc atchley -scale minmax -test True -plots True -lr 5e-5 -nb_epochs 1000 -batchsize 200 -outdir 'dataset4_xdmm_5feats'

python train.py -indir .\training_data_new\dataset4\ -arch xd2 -enc atchley -scale minmax -test True -plots True -lr 1e-4 -nb_epochs 1000 -batchsize 300 -outdir 'dataset4_xd2_mm_5feats'